### Mount Drive

In [ ]:
try:
    from google.colab import drive # type: ignore
    drive.mount('/content/home')
except ImportError:
    pass

Mounted at /content/home


### DIR and DEPENDENCIES

In [ ]:
import sys
import pathlib

colab_data = pathlib.Path('/content/home/MyDrive/B_SanLuis')
local_data = pathlib.Path("../data/")

DIR = colab_data if colab_data.exists() else local_data
sys.path.append(str(DIR / 'Scripts'))



CropConcern = 'Lettuce2024' #Change between Broccoli and Lettuce
latitude, longitude = 36.456667, 121.382222 #tower location
hemisphere = 'north'

if CropConcern == 'Broccoli2023' :
  in_df = DIR / '/Data/1_Intermediate_Datasets/Broccoli/Broccoli2023FP.csv'
  Json_out_df = DIR / '/Data/1_Intermediate_Datasets/Broccoli/Json/'
  out_df = DIR / '/Data/'

if CropConcern == 'Lettuce2023' :
  in_df = DIR / '/Data/1_Intermediate_Datasets/Lettuce/Lettuce2023FP.csv'
  Json_out_df = DIR / '/Data/1_Intermediate_Datasets/Lettuce/Json/'
  out_df = DIR / '/Data/'

if CropConcern == 'Lettuce2024' :
  in_df = DIR / '/Data/1_Intermediate_Datasets/Lettuce/Lettuce2024FP.csv'
  Json_out_df = DIR / '/Data/1_Intermediate_Datasets/Lettuce/Json/'
  out_df = DIR / '/Data/'


In [ ]:
#import dependencies
import src.calc_footprint_FFP as ffp  # noqa: E402
#import calc_footprint_FFP_climatology as ffp_clim
import pandas as pd  # noqa: E402
import pyproj as proj  # noqa: E402
import numpy as np  # noqa: E402
import geopandas as gpd  # noqa: E402
from shapely.geometry import Polygon # noqa: E402  
from shapely.ops import unary_union # noqa: E402
from shapely.geometry import Point # noqa: E402
from shapely.geometry import shape # noqa: E402
from shapelysmooth import taubin_smooth # noqa: E402
from pyproj import Transformer # noqa: E402
import rasterio # noqa: E402
from rasterio.transform import from_origin # noqa: E402
from rasterio.features import rasterize # noqa: E402
from rasterio.features import shapes # noqa: E402
from rasterio import features # noqa: E402
import matplotlib.pyplot as plt # noqa: E402

###Extract from FFP


In [6]:
# Calculate UTM zone and CRS based on tower location (set latlon in dependencies)
towercoordinates = np.array([[latitude, longitude]])  # tower's lat/lon
utmzone = int((longitude + 180) / 6) + 1  # calculate UTM zone based on longitude
utm_crs = f"+proj=utm +zone={utmzone} +datum=WGS84 +units=m +no_defs"  # create UTM CRS

# Transformer for lat/lon to UTM coordinates
transformer = Transformer.from_crs("epsg:4326", utm_crs)

# Function to convert lat/lon to UTM coordinates
def convert_to_utm(lat_lon):
    lat, lon = lat_lon
    easting, northing = transformer.transform(lat, lon)
    return easting, northing  # meters from bottom left of UTM zone

# Convert tower coordinates to UTM
tower_utm_coordinates = np.apply_along_axis(convert_to_utm, axis = 1, arr = towercoordinates)
easting, northing = tower_utm_coordinates[0]

In [ ]:
# Initialize an empty list to hold all the polygons
polygons = []

# Load crop data
crop_df = pd.read_csv(in_df)  # CSV contains 1048 rows

# Limit rows processed (for memory reasons) ~ 300 max
for index, row in crop_df.iterrows():
    if index >= 300:  # set to and # of rows
        break

    try:
        # Initialize the Footprint Model (FFP)
        TestFPmodel = ffp.FFP(
            zm = row['zm'],
            z0 = row['z0'],
            umean = row['u_mean'],
            h = 2000,
            ol = row['L'],
            sigmav = row['sigma_v'],
            ustar = row['u_star'],
            wind_dir = row['wind_dir'],
            rs = [90],
            fig = 0  # set to 1 if you want to plot the figure
        )

        # Extract xr, yr coordinates and shift them by tower's UTM coordinates
        xr = np.array(TestFPmodel['xr'][0]) + easting
        yr = np.array(TestFPmodel['yr'][0]) + northing

        # Create Polygon from xr, yr coordinate pairs
        polygon_coords = Polygon(zip(xr, yr))

        # Append the polygon to the list
        polygons.append(polygon_coords)

    except KeyError as e:
        print(f"KeyError in row {index}: {e}")
    except ValueError as e:
        print(f"ValueError in row {index}: {e}")
    except Exception as e:
        print(f"Error in row {index}: {e}")
        continue

# Create GeoDataFrame from all collected polygons at once
gdf = gpd.GeoDataFrame(geometry = polygons, crs = utm_crs)

# Optionally save the intermediates as GeoJSON
Json_output_path = Json_out_df + CropConcern + '_all.geojson'
gdf.to_file(Json_output_path, driver='GeoJSON')
print(f"All polygons saved as {Json_output_path}")


Error(0009):
 ustar (friction velocity) must be >=0.1.

Alert(0013):
 Using z0, ignoring umean if passed.
Error in row 0: invalid index to scalar variable.

Alert(0013):
 Using z0, ignoring umean if passed.
Error in row 1: invalid index to scalar variable.

Alert(0013):
 Using z0, ignoring umean if passed.
Error in row 2: invalid index to scalar variable.

Alert(0013):
 Using z0, ignoring umean if passed.
Error in row 3: invalid index to scalar variable.

Alert(0013):
 Using z0, ignoring umean if passed.
Error in row 4: invalid index to scalar variable.

Alert(0013):
 Using z0, ignoring umean if passed.
Error in row 5: invalid index to scalar variable.

Alert(0013):
 Using z0, ignoring umean if passed.
Error in row 6: invalid index to scalar variable.

Alert(0013):
 Using z0, ignoring umean if passed.
Error in row 7: invalid index to scalar variable.

Alert(0013):
 Using z0, ignoring umean if passed.
Error in row 8: invalid index to scalar variable.

Alert(0013):
 Using z0, ignoring u

###Polygon to Raster

In [ ]:
# Check if there is anything in gdf
if not gdf.empty:
    # Find furthest extent of polygons
    minx, miny, maxx, maxy = gdf.unary_union.bounds
    print(f"Overall extent: minx = {minx}, miny = {miny}, maxx = {maxx}, maxy = {maxy}")
else:
    print("No polygons were processed.")

In [ ]:
# Input desired resolution
resolution = 1  # meters per pixel
width = int((maxx - minx) / resolution)
height = int((maxy - miny) / resolution)

# Create array to hold all individual rasters
accumulated_raster = np.zeros((height, width), dtype=np.uint8)

# Set transformation matrix based on the bounding box
transform = from_origin(minx, maxy, resolution, resolution)

In [ ]:
# Iterate over all polygons in the GeoDataFrame and rasterize them
for index, row in gdf.iterrows():
    try:
        polygon = [row['geometry']]  # Convert polygon to list
        rasterized = features.rasterize(
            polygon,
            out_shape = (height, width),
            fill = 0,
            transform = transform
        )

        # Ensure rasterized data matches the dtype of the accumulated_raster
        rasterized = rasterized.astype(accumulated_raster.dtype)

        # Accumulate the rasterized polygon to the overall raster
        accumulated_raster += rasterized

    except KeyError as e:
        print(f"KeyError in row {index}: {e}")
    except Exception as e:
        print(f"Error in row {index}: {e}")
        continue

# Plot the accumulated raster with chosen colormap
fig, ax = plt.subplots()
# Set colormap (adjust as needed; 'hot' is currently used, but you can change this)
cax = ax.imshow(accumulated_raster, cmap = 'hot', extent = (minx, maxx, miny, maxy))

# Set labels and title
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
plt.colorbar(cax, ax = ax, label = 'Overlap Count')
plt.title('Accumulated Raster with Overlap')

# Display the plot
plt.show()

###Rid of zeroes

In [ ]:
# Identify all non-zero pixels and their values
non_zero_pixels = np.nonzero(accumulated_raster)  # Returns indices of non-zero pixels
non_zero_values = accumulated_raster[non_zero_pixels]  # Extract the actual values of non-zero pixels

# Get the corresponding coordinates (x, y) for each non-zero pixel
xs, ys = rasterio.transform.xy(transform, non_zero_pixels[0], non_zero_pixels[1])

# Create Point geometries from x, y coordinates
geometries = [Point(x, y) for x, y in zip(xs, ys)]

# Create a GeoDataFrame to hold the geometries and the overlap counts
accumulated_raster_non_zero = gpd.GeoDataFrame({
    'geometry': geometries,
    'overlap_count': non_zero_values
}, crs = utm_crs)


In [ ]:
# Plot the non-zero overlap pixels using GeoPandas
fig, ax = plt.subplots(figsize=(6, 6))
accumulated_raster_non_zero.plot(
    ax = ax,
    column = 'overlap_count',
    cmap = 'hot',
    legend = True,
    legend_kwds = {'label': "Overlap Count"}
)

# Label the axes and set the title
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
ax.set_title('Non-Zero Overlap Pixels')

# Display the plot
plt.show()

###Raster to Vector


In [ ]:
# Find the maximum overlap value directly inside the function
def raster_to_polygon(raster, transform, threshold_ratio = 0.8):
    max_overlaps = np.max(raster)  # Find max directly in the function
    mask = raster > (max_overlaps * threshold_ratio)  # Mask based on threshold (80% of max)
    shapes_gen = shapes(raster, mask = mask, transform = transform)  # Generate shapes

    # Convert shapes to polygons
    return [shape(geom) for geom, _ in shapes_gen]

In [ ]:
# Generate polygons from raster
raster_to_polygons = raster_to_polygon(accumulated_raster, transform)

# Merge the polygons into a single unified polygon
combined_vector_polygons = unary_union(raster_to_polygons)

# Create GeoDataFrame from the combined polygon
vector_gdf = gpd.GeoDataFrame(geometry = [combined_vector_polygons], crs = utm_crs)


# Smooth the polygon geometry, catching potential smoothing errors
try:
    vector_gdf.loc[0, 'geometry'] = taubin_smooth(vector_gdf.loc[0, 'geometry'], steps = 50)
except Exception as e:
    print(f"Error in smoothing polygon: {e}")
#the taubin_smooth function smoothes the geometry without changing the number of vertices

In [ ]:
# Plot the final smoothed polygon
fig, ax = plt.subplots(figsize = (6, 6))
vector_gdf.plot(ax = ax, edgecolor = 'black', facecolor = 'none')
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
plt.title('Polygon')

plt.show()

In [ ]:
# Save final
GeoJson_output_path = out_df + CropConcern + 'Final.geojson'
vector_gdf.to_file(GeoJson_output_path, driver = 'GeoJSON')

In [ ]:
vector_gdf

In [ ]:
vector_gdf['geometry'][0].area